# Qwen3(4B) 메뉴 조합 점수화 노트북 (Unsloth 스타일)

이 노트북은 **Qwen3-4B-Instruct**를 사용해 아래 4개 카테고리에서 1개씩 뽑은 메뉴 조합의 점수를 계산합니다.

- Appetizer
- Main Dish
- Drink
- Dessert

> Colab에서 바로 실행 가능한 형태로 구성했습니다.


In [ ]:
# Unsloth + Qwen3 권장 설치 (Colab)
# 기본 Qwen3/Unsloth 템플릿과 맞추기 위해 torch 계열을 먼저 정렬합니다.
!pip -q install torch==2.9.0 torchvision==0.24.0 torchaudio==2.9.0
!pip -q install unsloth transformers==4.52.4 accelerate bitsandbytes sentencepiece cuda-bindings==12.9.5


In [ ]:
import json
import random
import itertools
from typing import Dict, List

from unsloth import FastLanguageModel
import torch


In [ ]:
# 1) 사용자 지정 데이터셋
DATASET = {
    "appetizer": [
        "salad", "corn soup", "miso soup", "house bread", "cheese",
        "cracker", "Scotch Egg", "mashed potato", "nachos", "pasta"
    ],
    "main_dish": [
        "ramen", "pizza", "fried chicken", "sandwich", "T-bone steak",
        "sushi", "taco", "grilled tofu", "fish and chips", "paella"
    ],
    "drink": [
        "coca-cola", "red wine", "white wine", "sake", "green tea",
        "orange juice", "coke zero", "modelo beer", "coffee", "water"
    ],
    "dessert": [
        "orange", "grape", "pudding", "ice cream", "tart",
        "cheesecake", "macaron", "dango", "muffin", "churro"
    ]
}

print({k: len(v) for k, v in DATASET.items()})
print("총 가능한 조합 수:", len(DATASET["appetizer"]) * len(DATASET["main_dish"]) * len(DATASET["drink"]) * len(DATASET["dessert"]))


In [ ]:
# 2) Qwen3 로드 (Unsloth template)
max_seq_length = 2048
model_name = "unsloth/Qwen3-4B-Instruct-bnb-4bit"

model, tokenizer = FastLanguageModel.from_pretrained(
    model_name = model_name,
    max_seq_length = max_seq_length,
    dtype = None,      # 자동 선택
    load_in_4bit = True,
)
FastLanguageModel.for_inference(model)

print("Loaded:", model_name)


In [ ]:
SYSTEM_PROMPT = """You are a food pairing evaluator.
Given exactly one appetizer, one main dish, one drink, and one dessert,
return a strict JSON object with fields:
- score: integer from 0 to 100
- reason: short explanation in Korean
Scoring criteria:
1) Flavor harmony (40)
2) Texture balance (20)
3) Temperature/course flow (20)
4) Overall coherence (20)
Return ONLY JSON.
"""

def build_user_prompt(combo: Dict[str, str]) -> str:
    return (
        f"appetizer: {combo['appetizer']}\n"
        f"main_dish: {combo['main_dish']}\n"
        f"drink: {combo['drink']}\n"
        f"dessert: {combo['dessert']}"
    )


def parse_json_from_text(text: str):
    text = text.strip()
    start = text.find('{')
    end = text.rfind('}')
    if start == -1 or end == -1 or end <= start:
        return None
    chunk = text[start:end+1]
    try:
        data = json.loads(chunk)
        if isinstance(data.get("score"), (int, float)):
            data["score"] = int(data["score"])
        return data
    except Exception:
        return None


def evaluate_combo_with_qwen3(combo: Dict[str, str], max_new_tokens: int = 180):
    messages = [
        {"role": "system", "content": SYSTEM_PROMPT},
        {"role": "user", "content": build_user_prompt(combo)},
    ]

    inputs = tokenizer.apply_chat_template(
        messages,
        tokenize=True,
        add_generation_prompt=True,
        return_tensors="pt",
    ).to(model.device)

    with torch.inference_mode():
        outputs = model.generate(
            input_ids=inputs,
            max_new_tokens=max_new_tokens,
            do_sample=True,
            temperature=0.4,
            top_p=0.9,
            repetition_penalty=1.05,
        )

    generated = tokenizer.decode(outputs[0][inputs.shape[-1]:], skip_special_tokens=True)
    parsed = parse_json_from_text(generated)
    return parsed, generated


In [ ]:
# 3) 카테고리별 1개씩 랜덤으로 뽑아 점수 계산

def sample_combo(seed: int = None) -> Dict[str, str]:
    if seed is not None:
        random.seed(seed)
    return {
        "appetizer": random.choice(DATASET["appetizer"]),
        "main_dish": random.choice(DATASET["main_dish"]),
        "drink": random.choice(DATASET["drink"]),
        "dessert": random.choice(DATASET["dessert"]),
    }

combo = sample_combo(seed=42)
result, raw = evaluate_combo_with_qwen3(combo)

print("선택된 조합:")
print(combo)
print("\n모델 원문 출력:")
print(raw)
print("\n파싱 결과:")
print(result)


In [ ]:
# (옵션) 여러 조합 샘플링 후 점수 상위 보기

def evaluate_n_random(n: int = 20, seed: int = 0) -> List[Dict]:
    random.seed(seed)
    rows = []
    for _ in range(n):
        combo = sample_combo()
        parsed, raw = evaluate_combo_with_qwen3(combo)
        score = parsed.get("score") if parsed else None
        reason = parsed.get("reason") if parsed else raw
        rows.append({"combo": combo, "score": score, "reason": reason})

    rows = [r for r in rows if r["score"] is not None]
    rows.sort(key=lambda x: x["score"], reverse=True)
    return rows

ranked = evaluate_n_random(n=12, seed=7)
for i, row in enumerate(ranked[:5], 1):
    print(f"#{i} | score={row['score']} | {row['combo']}\n  reason={row['reason']}\n")
